 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [ ]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
from pathlib import Path
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_path = Path("Resources/whale_returns.csv")
whale_df = pd.read_csv(whale_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
whale_df.head()

In [ ]:
# Count nulls
whale_df.isnull().sum()

In [ ]:
# Drop nulls
whale_df.dropna(inplace=True)
whale_df

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_path = Path("Resources/algo_returns.csv")
algo_df = pd.read_csv(algo_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
algo_df.head()

In [ ]:
# Count nulls
algo_df.isnull().sum()

In [ ]:
# Drop nulls
algo_df.dropna(inplace=True)
algo_df.head()

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_hist = Path("Resources/sp500_history.csv")
sp500_hist = pd.read_csv(sp500_hist, index_col="Date", parse_dates=True, infer_datetime_format=True)
sp500_hist.head()

In [ ]:
# Check Data Types
sp500_hist.dtypes

In [ ]:
# Fix Data Types
sp500_hist["Close"] = sp500_hist["Close"].str.replace('$', '').astype("float")
sp500_hist.head()

In [ ]:
# Calculate Daily Returns
sp500_daily_returns = sp500_hist.pct_change()

In [ ]:
# Drop nulls
sp500_daily_returns.dropna(inplace=True)
sp500_daily_returns.head()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp500_daily_returns.columns = ["SP 500"]
sp500_daily_returns.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combined_df = pd.concat([whale_df, algo_df, sp500_daily_returns], axis="columns", join="inner")
combined_df.head()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_df.plot(figsize = (20,10), title = "Daily Returns")

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1 + combined_df).cumprod()

# Plot cumulative returns
cumulative_returns.plot(figsize = (20,10), title="Cumulative Returns")

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
combined_df.plot(kind ="box", figsize = (20,10), title="Risk")

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
whale_df_std = pd.DataFrame(whale_df.std())
whale_df_std

In [ ]:
algo_df_std = pd.DataFrame(algo_df.std())
algo_df_std

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500
sp500_daily_returns_std = pd.DataFrame(sp500_daily_returns.std())
sp500_daily_returns

In [ ]:
# Determine which portfolios are riskier than the S&P 500

#Tiger and Berkshire are riskier

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
#annual_std = combined_df.std() * np.sqrt(252)
annual_std = pd.DataFrame(combined_df.std()) * np.sqrt(252)
annual_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for the S&P 500 using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
# Plot the rolling standard deviation
combined_df.rolling(window=21).std().plot(figsize=(20, 10), title='Rolling 21-Day Standard Deviation of all Returns')

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation = combined_df.corr()
correlation
# Display de correlation matrix
sns.heatmap(correlation, vmin=-1, vmax=1)

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
covariance = combined_df['Algo 1'].rolling(window=60).cov(combined_df['SP 500'])
covariance
# Calculate variance of S&P 500
variance = combined_df['SP 500'].rolling(window=60).var()

# Computing beta
#amzn_beta = covariance / variance
beta = covariance / variance
#amzn_beta
# Plot beta trend
beta.plot(figsize=(20, 10), title='Rolling 60-Day Beta of Amazon')

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window
combined_df.ewm(halflife = 21).std().plot(figsize = (20,10))


# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = combined_df.mean()*252/(combined_df.std()*np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind = "bar", title = "Sharpe Ratios")

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

In [ ]:
# Algo 1 outperformed both the market and the whales porfolios.  Algo 2 underperformed the market and Berkshire but outperformed the others.

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [ ]:
# Reading data from 1st stock
goog_path = Path("Resources/goog_historical.csv")
goog_df = pd.read_csv(goog_path, parse_dates=True, infer_datetime_format=True)
goog_df.head()

In [ ]:
# Reading data from 2nd stock
aapl_path = Path("Resources/aapl_historical.csv")
aapl_df = pd.read_csv(aapl_path, parse_dates=True, infer_datetime_format=True)
aapl_df.head()

In [ ]:
# Reading data from 3rd stock
cost_path = Path("Resources/cost_historical.csv")
cost_df = pd.read_csv(cost_path, parse_dates=True, infer_datetime_format=True)
cost_df.head()

In [ ]:
# Combine all stocks in a single DataFrame
portfolio = pd.concat((goog_df, aapl_df, cost_df), axis = 1, join = "inner")
#portfolio.sort_index(ascending = True, inplace = True)
portfolio.head()

In [ ]:
# Reset Date index
portfolio.reset_index(drop=True)

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
portfolio_returns = portfolio.pct_change().dot(weights)
portfolio_returns.dropna(inplace=True)
# Display sample data
portfolio_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
joined_returns_df = pd.concat([portfolio_returns, combined_df], axis='columns', join='inner')

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
joined_returns_df.dropna(inplace=True)

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
total_portfolio_std = pd.DataFrame(portfolio_returns.std())
total_portfolio_std

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
rolling_std = portfolio_returns.rolling(window=21).std()
# Plot rolling standard deviation
rolling_std.plot(figsize = (20,10), title="21-Day Rolling Standard Deviations")

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
correlation = portfolio_returns.corr()
correlation

### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta
rolling_cov = portfolio_returns['GOOG/AAPL/COST'].rolling(window=60).cov(portfolio_returns['S&P 500'])

rolling_var = portfolio_returns['GOOG/AAPL/COST'].rolling(window=60).var()

rolling_beta = rolling_cov / rolling_var

rolling_beta.plot(figsize=(20, 10), title="Rolling 60-Day Beta")

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualzied Sharpe Ratios
portfolio_annual_sharpe_ratios = portfolio_returns.mean()*252/(portfolio_returns.std()*np.sqrt(252))

In [ ]:
# Visualize the sharpe ratios as a bar plot
portfolio_annual_sharpe_ratios.plot(kind = "bar", title = "Annual Sharpe Ratios")

### How does your portfolio do?

Write your answer here!

In [ ]:
# My portfolio outperformed the algorithms